<div>
<img src="webinar_promo.jpg" width="750"/>
</div>

# __Extract, Transform, Load (ETL) and Decline Curve Analysis (DCA)__


## __Self Introduction__________________________________________________________

My name is Elii Skeans and I am a reservoir engineer at Surtek. I hold a B.S. Petroleum Engineering from Colorado School of Mines, B.B.A. International Business from the University of Akron and a M.S. Data Analytics from CSU to be completed December 1st of this year.

I specialize in compositional numerical modeling for enhanced oil recovery in conventional and unconventional reservoirs, oilfield data analytics and economic modeling for oilfield valuation.

I have had the luxury in my psoition so far to perform reservoir simulation on world class assets in South America, Libya, Kuwait, Canada and the U.S.

<div><img src="Surtek_logo.png" width="300"/></div>

Since 1978 Surtek has been a global leader in enhanced oil recovery  
 - Experience includes all forms of EOR but specialty is in chemical EOR (liquids).
 - Laboratory services specialized in EOR, world recognition for EOR coreflooding expertise.  (250+ laboratory evaluations)
 - Client list includes nation oil companies, major and independent oil companies, and chemical manufacturers. (375+ clients in 25+ countries)
 - Most successful Chemical EOR hotspots initiated/assisted by Surtek (China, Wyoming, Canada, Kuwait, etc.). 
 - 30+ years in modeling of EOR technologies.  Compositional modeling specialists.

Surtek was founded by subject matter experts
 - Marathon was leader in EOR from the 1960s through 1980s. Harry Surkalo was the reservoir manager responsible for EOR project implementation at Marathon (focused on chemical and thermal EOR technologies). Many Marathon EOR team members at Surtek. 
 - Harry Surkalo and Malcolm Pitts are SPE IOR Pioneer Award recipients.

## __Problem Statement_______________________________________________________


Production data is one of the most common datasets that a reservoir engineer will encounter. Every asset I have worked on has had production data in different formats.

Depending on the type of analysis planned, different formats are needed:
 - FHF data for simulation software
 - Month normalized data for DCA
 - Rates vs. Cums
 - Etc.
 
Python is the perfect tool to complete the require transformations and code is often reusable.

### __Wyoming Oil & Gas Commission Field Data (WOGC)__
The example I have built for this exercise involves the extraction of field production data (at the individual well level) for a field in Wyoming called Hartzog Draw. 

The field was chosen for no specific reason otherthan that it is still active and has many oil-bearing wells. 

Field data is available at http://pipeline.wyo.gov/FieldMenu2.cfm on the WOGC website.

Unfortunately this data is painful to collect. The screenshot below shows that for each of the ~250 wells in the field I would have to check a box next to the well name. That is a mind numbing task that anyone would hope to avoid.

<div><br/><img src="WOGC_Field_Query.png" width="1000"/><br/></div>


### __There is a better option but it is reserved to only those who know Python or an equivalent tool__

There is an option to download all of the Wyoming's historic production data at the well level from the following link: http://pipeline.wyo.gov/urecordsMenu.cfm?Skip=%27Y%27&oops=ID69217

<div><br/><img src="WOGC_ALL_DL.png" width="200"/><br/></div>

The two datasets of interest are the well headers and the well production data. Download both to follow along. 

The first thing you will notice is that the production data comes in (4) CSV files. The next thing you will notice is that Excel crashes when you try to open it because each spreadsheet alone busts the row limit in Excel.

### This is the PERFECT Python example in that you cannot even use Excel if you wanted to. 

### __Loading Data______________________________________________________________________________

For those who ar enew to Python, Pandas is an absolutely essential package which utilizes dataframes which I will abbreviate to _df_ . 
A great tool for aggregating data is the _glob_ package. Using _glob_ I tell Python to cycle through each file that starts with '_Counties_'.

__*As a warning I will tell you that this bit of code takes upwards of 30 minutes to complete. I provide a slice of the dataframe that allows for quicker runtimes and less RAM.__


In [3]:
import glob, pandas as pd
## WARNING! This code will take upwards of 30 minutes to run due to the large file size

#Set the folde rlocation fo the unzipped production files
file='WYO/Counties*'
#Create an empty dataframe (df) for aggregation
dftotal=pd.DataFrame()
#Cycle through each file, read it into a dataframe and concatenate it into the aggregate df
for prd in glob.glob(file):
    df=pd.read_excel(prd)
    dftotal=pd.concat([dftotal,df])
#Print the shape of the aggregated df
dftotal.shape

(1490485, 54)

In [6]:
dftotal.head()

,APINO,COMPANY,YR,JAN_OIL,JAN_GAS,JAN_WATER,JAN_DAYS,FEB_OIL,FEB_GAS,FEB_WATER,...,NOV_GAS,NOV_WATER,NOV_DAYS,DEC_OIL,DEC_GAS,DEC_WATER,DEC_DAYS,API_RES,ResCode,Reservoir
0,105063,DNR OIL & GAS INC,1978,22,0,9,16,55,0,24,...,0,18,30,25,0,11,23,105063676,676,MUDDY
1,105063,DNR OIL & GAS INC,1979,25,0,8,22,29,0,10,...,0,8,30,35,0,8,31,105063676,676,MUDDY
2,105063,DNR OIL & GAS INC,1980,37,0,9,31,28,0,7,...,0,25,17,72,0,21,31,105063676,676,MUDDY
3,105063,DNR OIL & GAS INC,1981,25,0,8,24,37,0,14,...,0,16,30,29,0,11,22,105063676,676,MUDDY
4,105063,DNR OIL & GAS INC,1982,42,0,16,29,42,0,16,...,0,23,22,0,0,0,0,105063676,676,MUDDY


We can see from the dataframe's shape that there are 1,490,485 rows in the data set. Printing the head of the df shows us that it is not listed as continuous time series data. __You will notice that there are seperate columns for each month and each production fluid.__ This means that you cannot plot this data in the current format.

For this data to be useful it must be transformed. Doing so will explode the number of rows in the dataset. Each row currently represents a year, so __doing a transform will increase the number of rows by 12X.__

The next dataset we need is the well headers. This is a much more manageable data set with only 142k rows.

This data table contains the field name for each well which is missing form the production data set.

We will find the API numbers associated with the field and select only those wells from the total production data set.

In [132]:
# Load the Well Headers
dfwell=pd.read_excel('WYO/101220WH.DBF.xlsx')
dfwell

,APINO,COMPANY,WN,UNIT_LEASE,LEASE_NO,FIELD_NAME,HORIZ_DIR,LAND_TYPE,COUNTY,COUNTYTXT,...,STATUSDATE,CAPINO,FIRSTSPUD,FIRSTCOMP,BASIN,FORM2STAT,FORM2MON,FORM2YEAR,UNIT_CODE,RECDATE
0,105063,DNR OIL & GAS INC,1,GOVT WILCOX,WYW0167604,BIG HOLLOW,N,13,1,ALBANY,...,19950616.0,49-001-05063,19550513.0,19630624.0,LARAMIE,PR,8.0,2020.0,NaN,0.0
1,105081,DNR OIL & GAS INC,1,COUGHLIN GOVT,WYC028447-A,REX LAKE,N,13,1,ALBANY,...,19571230.0,49-001-05081,19571113.0,19571230.0,LARAMIE,PR,8.0,2020.0,NaN,0.0
2,105084,DNR OIL & GAS INC,5,LOUIS COUGHLIN,WYC028447C,REX LAKE,N,13,1,ALBANY,...,19361130.0,49-001-05084,19370603.0,19370617.0,LARAMIE,PR,8.0,2020.0,NaN,0.0
3,105114,TAP MANAGEMENT INC,4,PARKINSON,NaN,LITTLE LARAMIE,N,20,1,ALBANY,...,20101001.0,49-001-05114,19480813.0,19480906.0,LARAMIE,SI,7.0,2020.0,NaN,19480806.0
4,105128,RICHARDSON OPERATING CO,5,WILSON,WYC056175A,QUEALY,N,13,1,ALBANY,...,20161209.0,49-001-05128,19560625.0,19470717.0,LARAMIE,SI,7.0,2020.0,WYW180406X,19470409.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142509,2126817,NORTH SILO RESOURCES LLC,16-66 0112 13NH,BELLE FOURCHE E,NaN,WC,H,23,21,LARAMIE,...,20201006.0,49-021-26817,NaN,NaN,DENVER-CHEYENNE BASIN,NaN,NaN,NaN,NaN,20200922.0
142510,2126966,KAISER FRANCIS OIL CO,7-0904H,Falcon,NaN,SILO,H,23,21,LARAMIE,...,20201006.0,49-021-26966,NaN,NaN,DENVER-CHEYENNE BASIN,NaN,NaN,NaN,NaN,20201001.0
142511,2124370,NORTH SILO RESOURCES LLC,17-63-W5-8-1NH,HARDING RANCH,NaN,WC,H,23,21,LARAMIE,...,20201007.0,49-021-24370,NaN,NaN,DENVER-CHEYENNE BASIN,NaN,NaN,NaN,NaN,20200930.0
142512,2124386,NORTH SILO RESOURCES LLC,17-63-W7-18-1NH,HARDING RANCH,NaN,WC,H,23,21,LARAMIE,...,20201007.0,49-021-24386,NaN,NaN,DENVER-CHEYENNE BASIN,NaN,NaN,NaN,NaN,20200930.0


In [133]:
#subset the well header data by field of interest
df_HD=dfwell[dfwell['FIELD_NAME']=='HARTZOG DRAW']['APINO']

#subset the production data to only wells in field of interest
df_HD_prd=df[df['APINO'].isin(df_HD)]

# df_HD_prd.to_csv('HartzogDrawPRD.csv')

Now our production data set includes only wells in the Hartzog Draw field. The shape of the data frame is shown below and we can see it is now only 7,768 rows long and there are 359 wells included.

In [22]:
print("Rows:  %s"%df_HD_prd.shape[0])
print("Wells: %s"%len(df_HD_prd.APINO.unique()))

Rows:  7768
Wells: 359


### Now for the Transform________________________________________________________________________
The main tool used here is _melt_ which is included in the Pandas package. 

<font color=blue>_id vars_</font> defines the columns that will remain columns and <font color=blue>_value vars_</font> are the column you want covnerted into rows.<font color=blue>_var name_</font> sets the name of the new row data and <font color=blue>_value name_</font> sets the name of the newly created column.

This tool is used to create columns for a new dataframe called _dfnew_ which is then reloaded into _df\__HD\__prd_

In [1]:
import pandas as pd

#This is a pre-parsed dataframe to reduce runtime for the presentation
df_HD_prd=pd.read_csv('HartzogDrawPRD.csv')

#set the columns you want to keep and create an empty dictionary for the creation of the relevent oil, water, gas columns
id_vars, dct=['APINO','YR',],{}
#create a column name list for each variable oil, water, and gas
for x in ['OIL','GAS','WATER']:
    dct['value_vars_%s' % x] = ['JAN_%s'%x, 'FEB_%s'%x, 'MAR_%s'%x, 'APR_%s'%x, 'MAY_%s'%x, 'JUN_%s'%x, 'JUL_%s'%x, 'AUG_%s'%x, 'SEP_%s'%x, 
                'OCT_%s'%x, 'NOV_%s'%x, 'DEC_%s'%x]
    
#create a new dataframe that transforms the original dataset into continuous timeseries data
dfnew = pd.DataFrame({
    'APINO':pd.melt(df_HD_prd, id_vars=id_vars,value_vars=dct['value_vars_OIL'],var_name="Month_val", value_name="Oil")['APINO'],
    'YR':pd.melt(df_HD_prd, id_vars=id_vars,value_vars=dct['value_vars_OIL'],var_name="Month_val", value_name="Oil")['YR'],
    'Month_val':pd.melt(df_HD_prd, id_vars=id_vars,value_vars=dct['value_vars_OIL'],var_name="Month_val", value_name="Oil")['Month_val'],
    'Oil':pd.melt(df_HD_prd, id_vars=id_vars,value_vars=dct['value_vars_OIL'],var_name="Month_val", value_name="Oil")['Oil'],
    'Gas':pd.melt(df_HD_prd, id_vars=id_vars,value_vars=dct['value_vars_GAS'],var_name="Month_val", value_name="Gas")['Gas'],
    'Water':pd.melt(df_HD_prd, id_vars=id_vars,value_vars=dct['value_vars_WATER'],var_name="Month_val", value_name="Water")['Water']
})

#The months column needs to be combined with the year column to create a datetime variable
#The dictionary below is used to map the month abbreviations to numeric values
month_transform={'JAN':1, 'FEB':2, 'MAR':3, 'APR':4, 'MAY':5, 'JUN':6, 'JUL':7, 'AUG':8, 'SEP':9,'OCT':10, 'NOV':11, 'DEC':12}
#Some cleaning of the month column and mapping to numeric values is done here
dfnew['Month_val']=dfnew['Month_val'].replace(to_replace='_OIL', value='', regex=True).map(month_transform)
#A new datetime column is created by combining the month and year data together
dfnew['Date']=pd.to_datetime((dfnew.YR*10000+dfnew.Month_val*100+1).apply(str),format='%Y%m%d')
#drop unwanted columns post transformation
df_HD_prd=dfnew.drop(['Month_val','YR'],axis=1)
df_HD_prd.head()

,APINO,Oil,Gas,Water,Date
0,530959,240,24,1922,1984-01-01
1,523836,1127,545,0,1978-01-01
2,523836,609,854,0,1979-01-01
3,523836,205,230,0,1980-01-01
4,523836,318,209,0,1981-01-01


__Now that is the format we are all used to and is easy to work with.__

This completes the ETL protion of the webinar. The idea here was to show how Python can save you time and effort in loading, manipulating and transforming data into formats that are useable in your work flow. Each data transformation is unique requiring the development of unique code. This example should show some of the basic functionalities of Pandas in Python.

# Decline Curve Analysis (DCA)

If you are watching this webinar then surely you are familiar with Arp's equation for decline curve analysis. This is the most basic tool used in estimating the future performance of a well. 

___Arp's Equation for Hyperboic Decline___

$ q=q_i*(1+btD_i)^\frac{-1}{b}$

Typically DCA is done using commercial software. That software is expensive and alot of smaller companies limit their access to these tools due to cost especially with the market today. This workflow is more to showcase the capabilities of Python and how it can boost your workflow to the next level. 

In this section I'll introduce two new packages that we'll combine together in this analysis; __Plotly__ and __SciPy__.

<div><br/><img src="SciPy.png" width="200"/><div><img src="plotly.png" width="200"/><br/></div>



Plotly is a package for making interactive plots. We are going to be using an extension called _plotly.express_ which still blows my mind at how easy it is to use. In the next section I make an interactive scatter plot in one line using 46 characters.

SciPy is an essential package in Python that acts as an ecosystem for hundreds of pre-defined mathematical operations typical in science and engineering. Optimization is listed as one of the main competencies of SciPy along with numerical integration, interpolation, linear algebra, and statistics.

We want to create a function that fits a decline curve using Arp's equation. We also want to be able to view the decline fit in real time and manipulate the variables as needed. So Plotly will show us the results and let us interact with them and SciPy will run code to generate a best fit to our data.

### Plotting______________________________________________________________________________
<font color=red>Let's make a simple plot of the production of some random well.</font>

In [296]:
import plotly.express as px

#choosing a random well
api=528309

import plotly.graph_objects as go

HD_wells['Class_Color']=HD_wells['WELL_CLASS'].map({'I':'cyan','O':'green','MW':'black','D':'cyan'})
HD_wells_sub=HD_wells[HD_wells['APINO']==api]
fig = go.Figure(go.Scattermapbox(
    fill = 'none',
    lon = HD_wells['LON'], lat = HD_wells['LAT'],
    marker = { 'size': 6 ,'color':HD_wells['Class_Color']},
hovertext=HD_wells['APINO']))

fig.update_layout(
    mapbox = {
        'style': "stamen-toner",
        'center': {'lon': HD_wells['LON'].mean(), 'lat': HD_wells['LAT'].mean() },
        'zoom': 9.25},
    showlegend = False)
fig2 = go.Figure(go.Scattermapbox(
    fill = 'none',
    lon = HD_wells_sub['LON'], lat = HD_wells_sub['LAT'],
    marker = { 'size': 10 ,'color':'red'}))
fig.add_trace(fig2.data[0])

fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.show()


#subset field production to get single well production data
dca_prd=df_HD_prd[df_HD_prd['APINO']==api]

#use plotly to create a simple scatter plot
fig2 = px.scatter(dca_prd, x="Date", y="Oil")
fig2.show()

C:\Users\es422_h69khbs\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Great. Now it looks like there has been some recompletions or well workover done around 2017 so lets grab a decline off of that.

To do so we'll have to subset the data. I'll just manually enter a date as a variable in prewritten code.

In [230]:
#Choose date range for dca fit
date_range='4/1/2017'
#grab data only after the cut off date and drop any zero values
dca_prd_sub=dca_prd[(dca_prd['Date']>date_range)&(dca_prd['Oil']>0)]

#re-plot the data
fig = px.scatter(dca_prd_sub, x="Date", y="Oil")
fig.show()

### Fitting the Data_________________________________________________________________________
Now we will create a function that contains the various permutations of Arp's equation. I want my function to be able to fit an exponential, hyperbolic, or harmonic decline

In [231]:
from scipy.optimize import curve_fit
import math, plotly.graph_objects as go


def dca(t,qi,b,d):
    if b==0:
        return qi*math.exp(-1*d*t) #exponential
    elif b==1:
        return qi/(1+d*t) #harmonic
    elif b>=1:
        return 0 #limiting b to values equal to or less than 1
    else:
        return qi*((1+b*d*t)**(-1/b)) #exponential

With the function built we can load the production data into the algorithm.

Below I sort the df by date, then reset the index which creates a new index from 0 to len(df). Now my x values are the df.index numbers and my y data is the oil rate. You must provide a starting point for the optimization variables to the algorithm which is what I am doing with <font color=red>_p0</font>_

The optimize function outputs (2) arrays. One contains the fitted values for variables and the other is a covariance array. I'll print the <font color=red>popt</font> list to see the values for <font color=blue>b</font>, <font color=blue>Qi</font>, and <font color=blue>Di</font>.

In [232]:
#Sort the dataframe and reset index to be numerically increasing
dca_prd_sub=dca_prd_sub.sort_values('Date').reset_index().drop(['index'],axis=1)
#Set the x and y data for the function
xdata,ydata=dca_prd_sub.index,dca_prd_sub.Oil

#This is your inital guess for the algorithm, here I am choosing the highest rate for Qi, b=0.5 and Di = 0.1/month
p0=[ydata[0],0.5,0.1]

#The equation outputs the estimated variable values in popt and a covariance array in pcov
popt, pcov = curve_fit(dca, xdata, ydata,p0=p0)

#Let's see what we predicted here
print('Qi, bbl/month: %.0f'%popt[0])
print('b, unitless:   %.2f'%popt[1])
print('Di, 1/month:   %.4f'%popt[2])

Qi, bbl/month: 1217
b, unitless:   1.00
Di, 1/month:   0.0953


Neat. Now I want to plot all of this. First lets create a new dataframe for the fit data. W'ell pass the fitted variables into the dca function to create a decline that goes out 20 years into the future. We will also set an economic limit, below this number the well will be shut in.

We might as well output the expected remaining production fro the well to assign a PDP value as well.

In [233]:
#Set an economic limit in bbl/month
economic_limit=150

dca_fit=pd.DataFrame({'Month':range(0,max(xdata)+240,1)})
dca_fit['Qcalc']=dca(dca_fit.Month,popt[0],popt[1],popt[2])
#delete values below artifical limit
dca_fit=dca_fit[(dca_fit['Qcalc']>150)]

#calculate the pdp to economic limit
pdp=dca_fit['Qcalc'][dca_fit.Month>dca_prd_sub.index.max()].sum()

#Plot each dataset seperately
fig=go.Figure()
fig.add_trace(go.Scatter(x=xdata,y=ydata,mode='markers'))
fig.add_trace(go.Scatter(x=dca_fit.Month, y=dca_fit.Qcalc))
fig.show()
print('PDP, bbl:      %.0f' %pdp)
print('Qi, bbl/month: %.0f'%popt[0])
print('b, unitless:   %.2f'%popt[1])
print('Di, 1/month:   %.4f'%popt[2])

PDP, bbl:      7040
Qi, bbl/month: 1217
b, unitless:   1.00
Di, 1/month:   0.0953


In [294]:
dates=[]
for month in dca_fit.index:
    dates.append(dca_prd_sub.Date.min()+relativedelta(months=+month))
dca_fit['Date']=dates


fig=go.Figure()
fig.add_trace(go.Scatter(x=dca_prd.Date,y=dca_prd.Oil,mode='markers'))
fig.add_trace(go.Scatter(x=dca_fit.Date, y=dca_fit.Qcalc))
fig.show()
print('PDP, bbl:      %.0f' %pdp)
print('Qi, bbl/month: %.0f'%popt[0])
print('b, unitless:   %.2f'%popt[1])
print('Di, 1/month:   %.4f'%popt[2])

PDP, bbl:      7040
Qi, bbl/month: 1217
b, unitless:   1.00
Di, 1/month:   0.0953


In [222]:

from dateutil.relativedelta import relativedelta

apis=[528137,528244,528247,562387,528136]
temp=df_HD_prd
active_wells=temp[(temp.Date=='7/1/2020') & (temp.Oil>0)]['APINO'].unique()
temp=df_HD_prd[df_HD_prd['APINO'].isin(apis)]
temp2=pd.DataFrame({'Date':temp.Date.unique()}).sort_values('Date').set_index('Date')

max_date=dca_prd.Date.max()+relativedelta(months=+240)
dcas=pd.DataFrame({'Date':pd.date_range(start=dca_prd.Date.min(),end=max_date).to_pydatetime().tolist()})
dcas=dcas[dcas['Date'].dt.day==1].set_index('Date')

for api in temp.APINO.unique():
    temp3=temp[temp['APINO']==api].filter(['Oil','Date']).set_index('Date').rename(columns={'Oil':api})
    temp2=pd.concat([temp3,temp2])


for api in temp.APINO.unique():
    dca_prd=df_HD_prd[(df_HD_prd['APINO']==api)]
    dca_prd_sub=dca_prd.sort_values('Date').reset_index().drop(['index'],axis=1)
    xdata=dca_prd_sub.index
    ydata=dca_prd_sub.Oil

    p0=[ydata.max(),0.5,0.1]
    popt, pcov = curve_fit(dca, xdata, ydata,p0=p0)

    dca_fit=pd.DataFrame({'Month':range(0,max(xdata)+240,1)})
    dca_fit['Qcalc']=dca(dca_fit.Month,popt[0],popt[1],popt[2])
    dca_fit=dca_fit[(dca_fit['Qcalc']>90)]
    pdp=dca_fit['Qcalc'][dca_fit.Month>max(xdata)].sum()
    try:
        max_date=dca_prd_sub.Date.min()+relativedelta(months=+dca_fit.index.max())
        dftemp=pd.DataFrame({'Date':pd.date_range(start=dca_prd_sub.Date.min(),end=max_date).to_pydatetime().tolist()})
        dftemp=dftemp[dftemp.Date.dt.day==1].reset_index().drop('index',axis=1)
        dftemp=dftemp.join([dca_fit]).set_index('Date').rename(columns={'Qcalc':api}).drop(columns=['Month'])
        dcas=dcas.join(dftemp)
    except:
        pass
dcas=dcas.join(temp2,lsuffix='_fit', rsuffix='_og')

for api in temp.APINO.unique():
    temp3=temp[temp['APINO']==api].filter(['Oil','Date']).set_index('Date').rename(columns={'Oil':api})
    temp2=pd.concat([temp3,temp2])

wells=temp.APINO.unique()

traces,updatemenu,buttons = [],[],[]
for series in ['fit','og']:
    for i in range(0,len(wells)):
        traces.append(go.Scatter(x=dcas.index,y=dcas[str(wells[i])+'_%s'%series],mode='markers',visible=True))

fig = go.Figure(data=traces)

# button with one option for each dataframe
i=0
for col in wells:
    viz,viz[i],viz[i+len(wells)]=[False]*2*len(wells),True,True
    rem=dcas[dcas.index>'7/1/2020'][str(col)+'_fit'].drop_duplicates().reset_index().drop(columns=['Date'],axis=1)
    rem['PVbbl']=(1/(1.1**(rem.index/12)))*rem[str(col)+'_fit']
    pdp,pvpdp=rem.sum()[0],rem.sum()[1]
    buttons.append(dict(method='update',label=str(col),args=[{'visible':viz},
                        {'title':str(col)+' <br>Rem. PDP, bbl:       %0.0f<br>Disc. Rem. PDP, bbl: %0.0f'%(pdp,pvpdp)}],))
    i+=1
# some adjustments to the updatemenus
updatemenu,your_menu = [] , dict()
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True

# add dropdown menus to the figure
fig.update_layout(showlegend=True, updatemenus=updatemenu)
fig.show()
# wells